### Imports

In [ ]:
from transformers import pipeline,BartTokenizer,AutoTokenizer,AutoModelForQuestionAnswering,AutoModelForSeq2SeqLM
import requests,re,nltk
from trl import PPOConfig,PPOTrainer
from os.path import exists

### Preprocessing

In [ ]:
def extract_text(html):
    text = re.sub(r'<[^>]+>|[<>\'"]|[^a-zA-Z0-9,.?:]', '', html)
    return text
user_query = input("Enter your query: ")
search_url = f"https://www.google.com/search?q={"+".join(user_query.split(" "))}"
response = requests.get(search_url,headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:135.0) Gecko/20100101 Firefox/135.0"
})
page_text = extract_text(response.text)

### Summarization

In [ ]:
prompt = (
    f"extract the information relevant to the query '{user_query}'"
    f"from the following text:\n\n{page_text}"
)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
summarizer = pipeline("summarization",model="facebook/bart-large-cnn")
summary = summarizer(prompt,max_length=200,min_length=50,do_sample=False)

In [ ]:
def chunk_text(text,tokenizer,max_tokens=512):
    sentences = nltk.sent_tokenize(text,language="english")
    chunks = []
    current_chunk = []
    current_length = 0
    for sentence in sentences:
        sentence_tokens = tokenizer.tokenize(sentence,truncation=False)
        sentence_token_count = len(sentence_tokens)
        if current_length + sentence_token_count <= max_tokens:
            current_chunk.append(sentence)
            current_length += sentence_token_count
        else:
            if current_chunk:
                chunks.append(" ".join(current_chunk))
                current_chunk = [sentence]
                current_length = sentence_token_count
            else:
                chunks.append(sentence[:500])
                current_chunk = []
                current_length = 0
    if current_chunk: chunks.append(" ".join(current_chunk))
    return chunks
def summarize_large_text(text,model_name="facebook/bart-large-cnn",max_chunk_tokens=512):
    tokenizer = BartTokenizer.from_pretrained(model_name)
    summarizer = pipeline("summarization",model=model_name)
    chunks = chunk_text(text,tokenizer,max_chunk_tokens)
    summaries = []
    for chunk in chunks:
        summary = summarizer(
            chunk,
            max_length=200,
            min_length=50,
            do_sample=False,
            truncation=True
        )[0]["summary_text"]
        summaries.append(summary)
    final_summary = " ".join(summaries)
    while len(tokenizer.tokenize(final_summary)) > 512:
        final_summary = summarizer(
            final_summary,
            max_length=512,
            min_length=100,
            do_sample=False,
            truncation=True
        )[0]["summary_text"]
    return final_summary

### Q&A

In [ ]:
sumry = summarize_large_text(page_text)
qa_pipeline = pipeline("question-answering",model="distilbert-base-cased-distilled-squad")
qa_result = qa_pipeline(question=user_query,context=sumry)

In [ ]:
print(qa_result["answer"])

### RLHF

In [ ]:
def user_feedback():
    feedback = input("Please rate the answer on a scale from 1 (poor) to 5 (excellent): ")
    try: return int(feedback)
    except ValueError: return 3
def compute_reward(fbk):
    return (fbk - 3) / 2

In [ ]:
summarizer_model_path = "../models/fine_tuned_bart"
if exists(summarizer_model_path):
    model_summarizer = AutoModelForSeq2SeqLM.from_pretrained(summarizer_model_path)
    tokenizer_summarizer = BartTokenizer.from_pretrained(summarizer_model_path)
else:
    model_summarizer = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
    tokenizer_summarizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

ppo_config_sum = PPOConfig(learning_rate=4e-5,batch_size=1,mini_batch_size=1)
ppo_trainer_sum = PPOTrainer(ppo_config_sum, model_summarizer, tokenizer_summarizer)

qa_model_path = "../models/fine_tuned_qa"
if exists(qa_model_path):
    model_qa = AutoModelForQuestionAnswering.from_pretrained(qa_model_path)
    tokenizer_qa = AutoTokenizer.from_pretrained(qa_model_path)
else:
    model_qa = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")
    tokenizer_qa = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")

ppo_config_qa = PPOConfig(learning_rate=4e-3,batch_size=1,mini_batch_size=1)
ppo_trainer_qa = PPOTrainer(ppo_config_qa,model_qa,tokenizer_qa)

In [ ]:
print("\nFeedback for Summary:")
fbk_sum = user_feedback("Rate the summary extraction (1-5): ")
reward_sum = compute_reward(fbk_sum)

print("\nFeedback for QA:")
fbk_qa = user_feedback("Rate the QA answer (1-5): ")
reward_qa = compute_reward(fbk_qa)

ppo_trainer_sum.step([prompt], [sumry], [reward_sum])
model_summarizer.save_pretrained(summarizer_model_path)
tokenizer_summarizer.save_pretrained(summarizer_model_path)

qa_prompt = f"Question: {user_query}\nContext: {sumry}"

ppo_trainer_qa.step([qa_prompt], [sumry], [reward_qa])
model_qa.save_pretrained(qa_model_path)
tokenizer_qa.save_pretrained(qa_model_path)